In [1]:
%load_ext autoreload
%autoreload 15


In [2]:
!pip3 install azure-cosmos

  Using cached https://files.pythonhosted.org/packages/6a/66/683228000d19273676f8bfc65a3881251cdd405674f692b8fa832b3e1aed/azure_cosmos-4.2.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/ee/ff/48bde5c0f013094d729fe4b0316ba2a24774b3ff1c52d924a8a4cb04078a/six-1.15.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/f8/4b/ea7faaafac956a168ab9a95a7ebe583f9d308e8332a68af0ed3128ef520c/azure_core-1.9.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/39/fc/f91eac5a39a65f75a7adb58eac7fa78871ea9872283fb9c44e6545998134/requests-2.25.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/f5/71/45d36a8df68f3ebb098d6861b2c017f3d094538c0fb98fa61d4dc43e69b9/urllib3-1.26.2-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/a2/38/928ddce2273eaa564f6f50de919327bf3a00f091b5baba8dfa9460f3a8a8/idna-2.10-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages

In [3]:
!pip3 install pdfminer.six

  Using cached https://files.pythonhosted.org/packages/93/f3/4fec7dabe8802ebec46141345bf714cd1fc7d93cb74ddde917e4b6d97d88/pdfminer.six-20201018-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/20/4d/a7046ae1a1a4cc4e9bbed194c387086f06b25038be596543d026946330c9/sortedcontainers-2.3.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/19/c7/fa589626997dd07bd87d9269342ccb74b1720384a4d739a1872bd84fbe68/chardet-4.0.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/7c/b6/1f3dd48a22fcd56f19e6cfa95f74ff0a64b046306354e1bd2b936b7c9ab4/cryptography-3.3.1-cp36-abi3-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/ee/ff/48bde5c0f013094d729fe4b0316ba2a24774b3ff1c52d924a8a4cb04078a/six-1.15.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/1c/1a/90fa7e7ee05d91d0339ef264bd8c008f57292aba4a91ec512a0bbb379d1d/cffi-1.14.4-cp36-cp36m-manylinux1_x86_64.whl
  Using cached https:/

In [4]:
import json
import re
from bs4 import BeautifulSoup
import os
import requests
from pprint import pprint
import uuid
from statistics import mode
from nltk.corpus import stopwords
from pdf2txt import convert_pdf
# !pip install tika
import tika
from definitions_v3 import *

/opt/app/RB/.local/lib/python3.6/site-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.2) or chardet (4.0.0) doesn't match a supported version!
  RequestsDependencyWarning)


In [5]:
pointsRE_heading=re.compile('(?:\s*\([a-z]{1,3}\)|[A-Z]{1}\s+[a-zA-Z0-9_\s]{5})')
stop_words = list(set(stopwords.words('english')))
y_cordinate_Re=re.compile('top:(\d{1,5})px')
height_re=re.compile('height:(\d{1,5})px')

In [6]:
def check_underline(positions,y_cord_text):
    positions=[i for i in positions if y_cord_text<=i< y_cord_text+16]
    if positions:
        return True
    else:
        return False
    

In [7]:
def get_pages_no(soup):
    total_items = int(soup.find('span', id='NewReleases_total').text)
    items_per_page = int(soup.find('span', id='NewReleases_end').text)
    return round(total_items/items_per_page)

In [8]:
from pdfminer.high_level import extract_pages

# print(len(list(extract_pages('test/Chubb16-250-1019 Chubb EBM Business Pack Product Disclosure Statement (PDS) and Policy Wording.pdf'))))

In [9]:
def get_underlines(soup):
    positions=[]
    for span in soup.find_all('span'):
        y_cord_list=y_cordinate_Re.findall(str(span))
        if y_cord_list:
            y_cord=int(y_cord_list[0])
        else:
            continue
        style="position:absolute; border: black 1px solid" in str(span)
        height_px_li=height_re.findall(str(span))
        if height_px_li:
            height_px=height_px_li[0]
        else:
            continue
        height=int(height_px)<15
        if all([style,height]):
            positions.append(y_cord)
    return list(set(positions))

In [10]:
def ext_extract(text_list,language):
    ext_flag=False
    if language=='english':
        ext_lis=['extension']
    else:
        ext_lis=['extensión','extensiones']
   
    ext_count=0
    position_list=[]
    position=0
    for text in text_list:
        # if any two of 'exclusion ,condition or extension is present in the text, that text is omitted from the logic'
        bool_list=['condition' in text.lower(),'exclusion' in text.lower(),'extension' in text.lower()]
        if sum(bool_list)>=2:
            continue
        for i in ext_lis:
            for match in re.finditer(i, text.lower()):
                position_list.append(position+match.start())               
                ext_count+=1
        position+=len(text)
    if ext_count>=1:
        ext_flag=True
    return(ext_flag,ext_count,position_list)

In [11]:
def excl_extract(text_list,language):
    exclusion_flag=False
    if language=='english':
        excl_lis=['exclud','not cover','except','does not mean','not includ','exclusion']
    else:
        excl_lis=['excepto','excepción','no in cluido','exclusión','excluidos','excluirlo','excluyendo','exclusiones','exclusion','excluyen','excluyentes']
   
    exclusion_count=0
    position_list=[]
    position=0
    for text in text_list:
        # if any two of 'exclusion ,condition or extension is present in the text, that text is omitted from the logic'
        bool_list=['condition' in text.lower(),'exclusion' in text.lower(),'extension' in text.lower()]
        if sum(bool_list)>=2:
            continue
        for i in excl_lis:
            for match in re.finditer(i, text.lower()):
                position_list.append(position+match.start())               
                exclusion_count+=1
        position+=len(text)
    if exclusion_count>=1:
        exclusion_flag=True
    return(exclusion_flag,exclusion_count,position_list)

In [12]:
def cond_extract(text_list,language):
    cond_flag=False
    if language=='english':
        cond_lis=['condition']
    else:
        cond_lis=['condición','condiciones','condicionado']
   
    cond_count=0
    position_list=[]
    position=0
    for text in text_list:
        # if any two of 'exclusion ,condition or extension is present in the text, that text is omitted from the logic'
        bool_list=['condition' in text.lower(),'exclusion' in text.lower(),'extension' in text.lower()]
        if sum(bool_list)>=2:
            continue
        for i in cond_lis:
            for match in re.finditer(i, text.lower()):
                position_list.append(position+match.start())               
                cond_count+=1
        position+=len(text)
    if cond_count>=1:
        cond_flag=True
    return(cond_flag,cond_count,position_list)

In [13]:
def font_extraction(soup):
    fontsizes=[]
    for divs in soup.findAll('div'):
        for j in divs.find_all('span'):
            ext_size=re.findall(r'font-size:(.*)px">',str(j))
            if ext_size:
                fontsizes.append(int(ext_size[0]))
    return(fontsizes)

In [14]:
def stopword_check(word,text,language):
    if language=='spanish':
        stop_words = list(set(stopwords.words('spanish')))
    else:
        stop_words = list(set(stopwords.words('english')))
        transition="although  instead  whereas  despite  conversely  otherwise  however moreover  likewise  comparatively  correspondingly  similarly  furthermore  additionallyver  rather  nevertheless  nonetheless  regardless  notwithstanding consequently  therefore  thereupon  forthwith  accordingly  henceforth"
        transition_words=transition.split()
        transition_words
        stop_words.extend(transition_words)

    if exclusion_check(text,language):
        return True
    if word.lower() in stop_words:
        return False
    else:
        return True

In [15]:
def exclusion_check(text,language):
    if language=='spanish':
        exclusion_bag=['excepto','excepción','no in cluido','exclusión','excluidos','excluirlo','excluyendo',' excluyen','exclusiones','exclusiones','exclusion','excluyentes']
    else:
        exclusion_bag=['exclusion','excluded','not covered','will not cover','will not pay']
    for word in exclusion_bag:
        if word in text.lower():
            return True
    return False
    

In [16]:
def criteria_check(text,language):
    if language=='spanish':
        exclusion_bag=['condición','condiciones','condicionado']
    else:
        exclusion_bag=['condition','conditions']
    for word in exclusion_bag:
        if word in text.lower():
            return True
    return False
    

In [17]:
def ext_check(text,language):
    if language=='english':
        ext_bag=['extension']
    else:
        ext_bag=['extensión','extensiones']
    for word in ext_bag:
        if word in text.lower():
            return True
    return False
    

In [18]:
start=re.compile('.*\d{1,3}.*(?:\-|\.|–)')
def clean(text):
    text=text.replace('\n','').replace('\t',' ')
    text=start.sub('',text).strip()
    

    return(text)
def clean_pharses(phrases):
    phrases=[clean(i) for i in phrases if len(i)> 3]
    phrases=list(set(phrases))
    return phrases

In [19]:
def create_local_index(path,subdir,language,country):
    pointsRE_heading=re.compile('(?:\s*\([a-z]{1,3}\)|(?<![A-z])[A-Z]{1}\s+[a-zA-Z0-9_\s]{5})')
    pageNumRE=re.compile('Page\s*(\d{1,3})',re.IGNORECASE)
    neglect_def=['policy','insured','schedule']
    local_indexed={}
    previous_span=''
    bold_phrases_indexed={}
    for file in os.listdir(path):
        if file.endswith('.pdf'):
            print(file)
            global file_count
            file_count+=1
            
            print("COUNT",file_count)
            try:
                pg_no=len(list(extract_pages(os.path.join(path,file))))
            except:
                pg_no=0
            try:
                html=convert_pdf(os.path.join(path,file),'html')
            except:
                continue
            soup = BeautifulSoup(html, 'html5lib')
            underline_positions=get_underlines(soup)
            try:
                fontsizes=font_extraction(soup)
                file_font_size_mode=mode(fontsizes)
            except:
                file_font_size_mode=8
                
            text_para,text_plain_para='',''
            text_lis,text_plain_lis,bold_lis,page_list,=[],[],[],[]
            bold=False
            sub_page_def_list=[]
            try:
                definitions=def_extraction(os.path.join(path,file))
            except:
                continue
            definitions={key:value for key,value in definitions.items() if key.lower().strip() not in neglect_def}
            def_terms=list(definitions.keys())
#             print(len(def_terms),'LEN DEFINITIONS')
#             print(definitions)
            definition_text=''
            header_match_object=(0,'',False)
            cond_header_match_object=(0,'',False)
            ext_header_match_object=(0,'',False)
            second_category=False
            head_found=False
            def_flag=False
            cond_head_found=False
            ext_head_found=False
            single_page_head_found=False
            condition_text=''
            ext_text=''
            excl_text=''
            previous_pg_num=[]
            def_in_page=[]
            cond_single_page_head_found=False
            ext_single_page_head_found=False
            
            font_size=file_font_size_mode

            for divs in soup.findAll('div'):
                div_text_list=[span.text for span in divs.find_all('span') ]
                page_str=str(divs.find_all('a'))
                page_num_results=pageNumRE.findall(page_str)
                if page_num_results:
                    pagenum=page_num_results[0]
#                     print(pagenum)
                    if pagenum!=previous_pg_num:
                        sub_definitions_in_page=[]
                    previous_pg_num=pagenum
#                     definition_text=''
                    
                    #only for pdfs
                    if(int(pagenum) in [1,2]) and 'Wording' in file and pg_no >20:
#                     print('Continued on wording 2')
                        continue

                    if text_lis:
                        text_lis=text_lis[:-1]
                        text_para=''.join(text_lis)
                        text_plain_lis=text_plain_lis[:-1]
                        text_plain_para=''.join(text_plain_lis)
                        if(len(re.findall('\.',text_para))>(len(text_para)/3) or (len(re.findall('\d',text_para))>80 and int(pagenum)<4 and  pg_no>20 and 'content' in text_para.lower())) :
                            print('PASS')
                            print(text_plain_para)
                            text_para,text_plain_para=' ',' '
                            text_lis,text_plain_lis=[],[]
                            single_page_head_found=False
                            head_found=False
                            cond_head_found=False
                            ext_head_found=False
                            continue
                        
                        else:
                            bold_lis=clean_pharses(bold_lis)
                            pagenum=int(pagenum)-1
                            excl_flag,excl_count,excl_pos_lis=excl_extract(text_plain_lis,language)
                            cond_flag,cond_count,cond_pos_lis=cond_extract(text_plain_lis,language)
                            ext_flag,ext_count,ext_pos_lis=ext_extract(text_plain_lis,language)
                            if int(pagenum)==1:
                                if head_found or single_page_head_found or excl_count>0:
#                                     print(pagenum,True,'!!!')
                                    excl_flag=True
                                else:
                                    excl_flag=False
                            else:
                                if head_found or single_page_head_found:
#                                     print(pagenum,True,'!!!',excl_count)
                                    excl_flag=True
#                                     print(excl_flag,'EXCL')
#                                 else:
#                                     print(pagenum,excl_flag,'\n')
# #                             print(excl_text)
#                             print(condition_text)
                            if cond_head_found or cond_single_page_head_found or cond_count>0:
                                cond_flag=True
#                                 print('COND','TRUE')
                            else:
#                                 print('COND','FALSE')
                                cond_flag=False
    
                            if ext_head_found or ext_single_page_head_found or ext_count>0:
                                ext_flag=True
#                                 print('EXT','TRUE',pagenum)
                            else:
#                                 print('EXT','FALSE',pagenum)
                                ext_flag=False
#                             def_in_page=[{'name':k,'text':v[0] for k,v in definitions.items() if v[1]=pagenum}]
                            for term,defs in definitions.items():
#                                 print(defs[1])
                                if defs[1]==pagenum:  ###defs[1] is the pagenumber
                                    definition_text=definition_text+'          '+term+' '+defs[0]
                                    def_in_page.append({'name':term,'text':defs[0]})
#                                     def_in_page{'name':}
#                             print(definition_text)
                            if definition_text:
                                def_flag=True
                            
                            if def_flag:
                                def_search_flag=True
#                                 print('DEF','TRUE')
                            else:
#                                 print('DEF', 'FALSE')
                                def_search_flag=False   
                            
                            if pg_no>2:
                                endorsements=False
                            else:
                                endorsements=True
#                             print(endorsements,'ENDORSEMENTS!!!')
                            if text_para in local_indexed.keys():
                                text_para=text_para+' '
                            
                            local_indexed[text_para]=(file,pagenum,bold_lis,text_plain_para,excl_flag,excl_count,excl_pos_lis,subdir,sub_page_def_list,country,language,definition_text,def_search_flag,endorsements,excl_text,condition_text,cond_flag,cond_count,cond_pos_lis,def_in_page,ext_text,ext_flag,ext_count,ext_pos_lis)
#                             print(sub_page_def_list,'DEF LIST!!!!!1')
                            sub_page_def_list=[]
                            page_list.append(pagenum)
                            text_para,text_plain_para='',''
                            text_lis,text_plain_lis,bold_lis=[],[],[]
                            definition_text=''
                            cond_single_page_head_found=False
                            ext_single_page_head_found=False
                            condition_text=''
                            ext_text=''
                            excl_text=''
                            single_page_head_found=False
                            def_in_page=[]
                            def_flag=False
                for span in divs.find_all('span'):

                    bold=False
                    upper=False
                    bullet=False
                    def_flag=False
                    span_position=div_text_list.index(span.text)
                    
                    if "Bold" in str(span) or 'CIDFont+F3' in str(span):
                        bold_lis.append(span.text)
                        bold=True
                    if span.text.isupper():
                        upper=True
                    font_family_match=re.findall(r"font-family: b'(.*)';",str(span))
                    if font_family_match:
                        font_family=font_family_match[0]
                    else:
                        font_family=''
#                     print(font_family)
                    font_size_match=re.findall(r'font-size:(.*)px">',str(span))
                    if font_size_match:
                        font_size=int(font_size_match[0])
                    if pointsRE_heading.findall(span.text):
                        bullet=True
#@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
                    y_cord_list=y_cordinate_Re.findall(str(divs))
                    if y_cord_list:
                        y_cord=int(y_cord_list[0])
                    underlined_text=check_underline(underline_positions,y_cord)
                    if span.text.split('\n')[0]!='' and span.text.split('\n')[0]!=' ':
                        head_check_text=span.text.split('\n')[0]
                    elif len(span.text.split('\n'))>1:
                        head_check_text=span.text.split('\n')[1]
                    else:
                        head_check_text=''
##############################3  
                    if str(previous_span).endswith('<br/></span>'):
                        span_position=0
                    ##################INSERT FUNCTION####################z
                    #EXCLUSION 
                    if  span_position==0 and (bold or font_size>= file_font_size_mode +2 ) and (font_size> file_font_size_mode or upper) and 5<len(head_check_text.strip())<80:                    
                        if exclusion_check(head_check_text,language) and not head_found: 
                            head_found=True
                            header_match_object=(font_size,font_family,bold,upper,bullet)
                            First_category=True
                            second_category=False
                            single_page_head_found=True
#                             print('FOUND Heading for exclusion ...',span.text)
                        
                    
                        elif ((font_size,font_family,bold,upper,bullet)==header_match_object  or font_size>header_match_object[0] ) and head_found and not exclusion_check(head_check_text,language):
                            head_found=False
                            single_page_head_found=True
#                             print('FOUND Closure for exclusion ...',span.text)
#                     if 'CLAIMS CONDITIONS' in span.text:
#                         print ('^^^^^^^',(font_size,font_family,bold,upper,bullet),header_match_object, font_size>header_match_object[0] , head_found , not exclusion_check(head_check_text,language))
#                         print(pointsRE_heading.findall(span.text))
                    words_title=[word.istitle() for word in head_check_text.split() if stopword_check(word,head_check_text,language) and not word.isdigit() ]
                    if span_position==0 and all(words_title) and head_check_text.strip() not in definitions and  (len(words_title) >1 or underlined_text )and 5<len(head_check_text.strip())<80 :
#                         print('MMMMMMMMARARARA',span.text)

                        if exclusion_check(head_check_text,language) and  not head_found:
                            head_found=True
                            second_category=True
                            First_category=False
                            single_page_head_found=True
                            header_match_object=(font_size,font_family,bold,upper,bullet)
#                             print("FOUND heading type2 ....",span.text)
                        elif head_found and not exclusion_check(head_check_text,language) and second_category and ((font_size,font_family,bold,upper,bullet)==header_match_object or font_size > header_match_object[0] ) :
                            head_found=False
                            second_category=False
                            single_page_head_found=True
#                             print('FOUND closure type 2',span.text)
                    if head_found:
                        excl_text=excl_text+' '+span.text     
                     
                    #CONDITIONS
#                     if 'Condition' in span.text:
# #                         print('text:',str(span.text),'head check',head_check_text,'length of head check',len(head_check_text))
# #                         print(span_position==0,(bold or font_size>= file_font_size_mode +2 ), (font_size> file_font_size_mode or upper), 5<len(head_check_text.strip())<80)
# #                         print(font_size,file_font_size_mode,cond_head_found,criteria_check(head_check_text,language))
                    if  span_position==0 and (bold or font_size>= file_font_size_mode +2 ) and (font_size> file_font_size_mode or upper) and 5<len(head_check_text.strip())<80:                    
                        if criteria_check(head_check_text,language) and not cond_head_found: 
                            cond_head_found=True
                            cond_header_match_object=(font_size,font_family,bold,upper,bullet)
                            cond_First_category=True
                            cond_second_category=False
                            cond_single_page_head_found=True
#                             print('FOUND Heading for condition...',span.text)
                    
                        elif ((font_size,font_family,bold,upper,bullet)==cond_header_match_object or font_size>cond_header_match_object[0] ) and not criteria_check(head_check_text,language)  and cond_head_found:
                            cond_head_found=False
                            cond_single_page_head_found=True
#                             print('FOUND Closure for condition ...',span.text)
                
                    
                    cond_words_title=[word.istitle() for word in head_check_text.split() if stopword_check(word,head_check_text,language) and not word.isdigit() ]
                    if span_position==0 and all(cond_words_title) and head_check_text.strip() not in definitions and  (len(words_title) >1 or underlined_text )and 5<len(head_check_text.strip())<80 :
#                         print('MMMMMMMMARARARA',span.text)

                        if criteria_check(head_check_text,language) and  not cond_head_found:
                            cond_head_found=True
                            cond_second_category=True
                            cond_First_category=False
                            cond_single_page_head_found=True
                            cond_header_match_object=(font_size,font_family,bold,upper,bullet)
#                             print("FOUND heading type2 ....",span.text)
                        elif cond_head_found and not criteria_check(head_check_text,language) and cond_second_category and ((font_size,font_family,bold,upper,bullet)==cond_header_match_object or font_size > cond_header_match_object[0] ) :
                            cond_head_found=False
                            cond_second_category=False
                            cond_single_page_head_found=True
#                             print('FOUND closure type 2',span.text)

                    if cond_head_found==True:
                        condition_text=condition_text+' '+span.text
                            
                    
                    #####EXTENSIONS
                    #CONDITIONS
#                     if 'Extension' in span.text:
#                         print(span.text)
#                         print('text:',str(span.text),'head check',head_check_text,'length of head check',len(head_check_text))
#                         print(span_position==0,(bold or font_size>= file_font_size_mode +2 ), (font_size> file_font_size_mode or upper), 5<len(head_check_text.strip())<80)
#                         print(font_size,file_font_size_mode,cond_head_found,criteria_check(head_check_text,language))
                    if  span_position==0 and (bold or font_size>= file_font_size_mode +2 ) and (font_size> file_font_size_mode or upper) and 5<len(head_check_text.strip())<80:                    
                        if ext_check(head_check_text,language) and not ext_head_found: 
                            ext_head_found=True
                            ext_header_match_object=(font_size,font_family,bold,upper,bullet)
                            ext_First_category=True
                            ext_second_category=False
                            ext_single_page_head_found=True
#                             print('FOUND Heading for ext...',span.text)
                    
                        elif ((font_size,font_family,bold,upper,bullet)==ext_header_match_object or font_size>ext_header_match_object[0] ) and not ext_check(head_check_text,language)  and ext_head_found:
                            ext_head_found=False
                            ext_single_page_head_found=True
#                             print('FOUND Closure for ext ...',span.text)
                
                    
                    ext_words_title=[word.istitle() for word in head_check_text.split() if stopword_check(word,head_check_text,language) and not word.isdigit() ]
                    if span_position==0 and all(ext_words_title) and head_check_text.strip() not in definitions and  (len(words_title) >1 or underlined_text )and 5<len(head_check_text.strip())<80 :
#                         print('MMMMMMMMARARARA',span.text)

                        if ext_check(head_check_text,language) and  not ext_head_found:
                            ext_head_found=True
                            ext_second_category=True
                            ext_First_category=False
                            ext_single_page_head_found=True
                            ext_header_match_object=(font_size,font_family,bold,upper,bullet)
#                             print("FOUND heading type2 ext....",span.text)
                        elif ext_head_found and not ext_check(head_check_text,language) and ext_second_category and ((font_size,font_family,bold,upper,bullet)==ext_header_match_object or font_size > ext_header_match_object[0] ) :
                            ext_head_found=False
                            ext_second_category=False
                            ext_single_page_head_found=True
#                             print('FOUND closure type 2 ext',span.text)
# 
                    if ext_head_found==True:
                        ext_text=ext_text+' '+span.text
                            
    #                     else:
###############################             
                        
                    if font_size_match:
                        if font_size<=file_font_size_mode-1:
                            text_lis.append(' ')
                    text=span.text.replace('\n',' ')
                    text_plain=text.replace('\n',' ')
                    for term in def_terms:
                        if term.lower().strip() in text.lower() and definitions[term][0].strip() not in text and term not in sub_definitions_in_page:
#                             def_page.append(definitions[term])
#                             print(pagenum)
                            src_str  = re.compile(re.escape(term), re.IGNORECASE)
                            text=src_str.sub('###{}@@{}%%%'.format(term,definitions[term][0]),text)
#                             print(text,'\n',term,'TERM!!!!@#$%^&*&^%#$%^&')
                            sub_definitions_in_page.append(term)
                            sub_page_def_list.append({'name':term,'text':definitions[term][0]})
#                             print(definitions_in_page)
                            break
#                     print(definitions_in_page)

                    
                    text_lis.append(text)
                    text_plain_lis.append(text_plain)
                    previous_span=span
                    

            if text_lis:
                if(int(pagenum) in [1,2]) and 'Wording' in file and pg_no >20:
#                     print('Continued on wording 2')
                    continue

                text_lis=text_lis[:-1]
                text_para=''.join(text_lis)
                text_plain_lis=text_plain_lis[:-1]
                text_plain_para=''.join(text_plain_lis)
                if(len(re.findall('\.',text_para))>(len(text_para)/3) or (len(re.findall('\d',text_para))>80 and int(pagenum)<4 and pg_no>20 and 'content' in text_para.lower())) :
                    print('PASS')
                    print(text_plain_para)
                    text_para,text_plain_para=' ',' '
                    text_lis,text_plain_lis=[],[]
                    single_page_head_found=False
                    head_found=False
                    cond_head_found=False
                    ext_head_found=False
                    continue
                else:
                    bold_lis=clean_pharses(bold_lis)
                    if pagenum in page_list and pagenum!='1':
                        pagenum=int(pagenum)+1
                    excl_flag,excl_count,excl_pos_lis=excl_extract(text_plain_lis,language)
                    cond_flag,cond_count,cond_pos_lis=cond_extract(text_plain_lis,language)
                    ext_flag,ext_count,ext_pos_lis=ext_extract(text_plain_lis,language)
                    if int(pagenum)==1:
                        if head_found or single_page_head_found or excl_count>0:
#                             print(pagenum,True,'!!!',excl_count)
                            excl_flag=True
                        else:
                            excl_flag=False
                    else:
                        if head_found or single_page_head_found:
#                             print(pagenum,True,'!!!')
                            excl_flag=True
#                             print(excl_flag,'EXCL')
                        
                    #####################
                    if cond_head_found or cond_single_page_head_found or cond_count>0:
                        cond_flag=True
#                         print('COND','TRUE')
                    else:
                        cond_flag=False
                    
                    if ext_head_found or ext_single_page_head_found or ext_count>0:
                        ext_flag=True
#                         print('EXT','TRUE',pagenum)
                    else:
#                         print('EXT','FALSE',pagenum)
                        ext_flag=False
#                         print('COND','FALSE')
#                     page_def_list.append({'name':term,'text':definitions[term][0]}
#                     def_in_page=[{'name':k,'text':v[0]} for k,v in definitions.items() if v[1]==pagenum]  
#                     print(def_in_page,'!!!!!!!!!!!!!!!!!!!!!!!!!!!!!')
                    for term,defs in definitions.items():
                        if defs[1]==pagenum:  ###defs[1] is the pagenumber
                            definition_text=definition_text+'          '+term+' '+defs[0]
                            def_in_page.append({'name':term,'text':defs[0]})        
#                     print(definition_text)
                    if definition_text:
                        def_flag=True

                    if def_flag:
                        def_search_flag=True
#                         print('DEF','TRUE')
                    else:
                        def_search_flag=False
#                         print('DEF','FALSE')
                    
                    if pg_no>2:
                        endorsements=False
                    else:
                        endorsements=True
#                     print(endorsements,'ENDORSEMENTS!!!')        
                    if text_para in local_indexed.keys():
                        text_para=text_para+' '
                    local_indexed[text_para]=(file,pagenum,bold_lis,text_plain_para,excl_flag,excl_count,excl_pos_lis,subdir,sub_page_def_list,country,language,definition_text,def_search_flag,endorsements,excl_text,condition_text,cond_flag,cond_count,cond_pos_lis,def_in_page,ext_text,ext_flag,ext_count,ext_pos_lis)
#                     print(sub_page_def_list,'DEF LIST!!!!!1')
                    sub_page_def_list=[]
                    text_para,text_plain_para='','' 
                    text_plain_lis,text_lis,bold_lis=[],[],[]
                    definition_text=''
                    single_page_head_found=False
                    cond_single_page_head_found=False
                    ext_single_page_head_found=False
                    condition_text=''
                    ext_text=''
                    excl_text=''
                    def_in_page=[]
                    def_flag=False
    return(local_indexed)

In [20]:
def doc_pdf_path(path):
    file_list=os.listdir(path)
    for file in file_list:
        raw_filename=file[:file.rfind(".")]
        pdf_filename=raw_filename+'.pdf'
#         print(raw_filename,pdf_filename,'FILE!!')
        if pdf_filename in file_list:
            print('continued!!!!!!!!!!!!!')
            continue
        if file.endswith('doc') or file.endswith('docx'):
            print('processing!!!')
            wordToPdf(os.path.join(path,file))

In [21]:
# rootDir='V://COG//AU- Property//'
# for dirname,subdirlist,filelist in os.walk(rootDir):
#     print(dirname)
#     for subdir in subdirlist:
#         path=os.path.join(dirname,subdir)
# #         doc_pdf_path(path)

In [22]:
# import os
file_count=0
rootDir='Documents/AU-Casualty/'
country='australia'
local_indexed={}
language='english'
if 'spanish' in rootDir:
    language='spanish'
for dirname,subdirlist,filelist in os.walk(rootDir):
    for subdir in subdirlist:
        print(subdir, 'SUBDIR!!!!!!!!!!!!!')
        path=os.path.join(dirname,subdir)    
        local_indexed_subdir=create_local_index(path,subdir,language,country)
        local_indexed.update(local_indexed_subdir)

Other Carrier Wording SUBDIR!!!!!!!!!!!!!
Broker Wording SUBDIR!!!!!!!!!!!!!
Mining Operation Activities Exclusion (SC_Vertex_EXL_0915.MOA.1).pdf
COUNT 1
Tour Operators Exclusion (SC_Vertex_UMB_0915.TOUROE.1).pdf
COUNT 2
MTBE Exclusion (SC_GA_UMB_V3.MTBE.2).pdf
COUNT 3
Oil Industry Exclusion (SC_Vertex_GPL_1018.OIE.1).pdf
COUNT 4
Chubb02-148-1020 Gallagher Chubb Combined General Legal Liability Quote.pdf
COUNT 5
PASS
          Contents  Important Notices........................................................................................ 3  Financial Strength Rating ............................................................................. 4  Combined General Legal Liability Insurance Quote ......................................... 5  Endorsements ............................................................................................. 6  Privacy Statement ........................................................................................ 8  About Chubb in Australia.....

TSE and BSE Exclusion (SC_Vertex_GPL_1018.TSEBSE.1).pdf
COUNT 102
Personal Injury Volunteer Exclusion (SC_MMA_CGL_V3.0.PIVE.1).pdf
COUNT 103
Mould Fungus Exclusion (SC_Vertex_EXL_0915.MFE.1).pdf
COUNT 104
Molestation Exclusion (SC_WP_GPL_0911.MOL.1).pdf
COUNT 105
Principals Indemnity (Specified) Endorsement (SC_Chubb02930217.PRISP.1).pdf
COUNT 106
MMA Combined Liability Insurance Quote Template.pdf
COUNT 107
Animal Disease Exclusion (SC_GA_UMB_V3.ADE.2).pdf
COUNT 108
Specified Activities Exclusion (SC_Chubb02930217.SA.1).pdf
COUNT 109
Change of Business Description Endorsement (SC_GA_EXCESS_CGL_V3.CHBD.1).pdf
COUNT 110
Injury to Contractor Exclusion (SC_Vertex_EXL_0915.CONT.1).pdf
COUNT 111
Austbrokers Broadform Liability Insurance Policy Wording Template.pdf
COUNT 112
Statutory Liability Endorsement (SC_Chubb21120617.STATL.1).pdf
COUNT 113
Labour Hire Exclusion (SC_GA_UMB_V3.LABOURE.2).pdf
COUNT 114
Meat Rejection Exclusion (SC_Vertex_EXL_0915.MEATRE.1).pdf
COUNT 115
GPS Endorsement (

Personal and Corporate Data Breach and Cyber Attack Extension (SC_MMA_CGL_V3.0.PCDBCAE.1).pdf
COUNT 209
EMF Exclusion (SC_Vertex_UMB_0915.EMF.1).pdf
COUNT 210
Lead Exclusion (SC_MMA_CGL_V3.0_LDE.1).pdf
COUNT 211
Rip and Tear Exclusion (SC_WP_GPL_0911.RPTR.1).pdf
COUNT 212
Change of Insured Endorsement (SC_GA_UMB_V3.CIE.1).pdf
COUNT 213
Diacetyl Exclusion (SC_MMA_CGL_V3.0.DE.1).pdf
COUNT 214
Pharmaceutical Herbaceutical Products Exclusion (SC_Vertex_EXL_0915.PRMHRB.1).pdf
COUNT 215
Design Exclusion (SC_GA_EXCESS_CGL_V3.DE.1).pdf
COUNT 216
Pharmaceuticals (Herbaceutical) Products Exclusion (SC_Chubb21120617.PHARHERB.1).pdf
COUNT 217
Specified Products Exclusion (SC_WP_GPL_0911.SPPE.1).pdf
COUNT 218
Rip and Tear Endorsement (SC_ Chubb21120617.RPTRE.1).pdf
COUNT 219
Principals Indemnity (Specified) Endorsement (SC_Chubb21120617.PRISP.1).pdf
COUNT 220
MTBE Exclusion (SC_WP_GPL_0911.MTBE.1).pdf
COUNT 221
Products Recall Expenses Endorsement (SC_Chubb21120617.PRORE.1).pdf
COUNT 222
Formaldehy

Biological Agents Exclusion (SC_Vertex_UMB_0915.BIOAE.1).pdf
COUNT 326
Demolition and Underpinning Exclusion (SC_MMA_CGL_V3.0_DEMUN.1).pdf
COUNT 327
Contractual Liability (Nominated Contract) Endorsement (SC_Vertex_UMB_0915.NCE.1).pdf
COUNT 328
Lead Exclusion (SC_Vertex_GPL_1018.LEAD.1).pdf
COUNT 329
CYBER LIABILITY EXCLUSION (SC_MMA_CGL_V3.0_CYLE.2).pdf
COUNT 330
Blood and Blood Bourne Products Exclusion (Write back for First Aid) (SC_Chubb02930217.BBBPE.1).pdf
COUNT 331
MTBE Exclusion (SC_Chubb21120617.MTBEE.1).pdf
COUNT 332
Human Clinical Trials Exclusion (SC_GA_EXCESS_CGL_V3.HUMCTE.1HUMCTE.1).pdf
COUNT 333
Defamation and Privacy Exclusion (SC_MMA_CGL_V3.0.DPE.1).pdf
COUNT 334
Statutory Liability Endorsement (SC_WP_GPL_0911.STATL.1).pdf
COUNT 335
MMA Excess Liability Policy (GA_Excess_CGL_V3).pdf
COUNT 336
Design Exclusion (SC_Vertex_UMB_0915.DE.1).pdf
COUNT 337
5. Extended Definition of Property Damage - Deterioration or Damage (SC_WP_GPL_0911.DEFD.1).pdf
COUNT 338
Human Clinical T

Additional Insured Endorsement (SC_Vertex_UMB_0915.ADDIE.1).pdf
COUNT 411
Specified Activities Exclusion (SC_Chubb02940217.SA.1).pdf
COUNT 412
Audience Participation Exclusion (SC_GA_UMB_V3.AUPART.2).pdf
COUNT 413
Lead Exclusion (SC_Chubb02930217.LDE.1).pdf
COUNT 414
Airside Liability Exclusion (SC_Chubb21120617.AIRLE.1).pdf
COUNT 415
Vehicle (Driving Risk) Exclusion (SC_Vertex_GPL_1018.VEHDR.1).pdf
COUNT 416
Organophosphates Exclusion (SC_Vertex_EXL_0915.OGPHO.1).pdf
COUNT 417
Change of Business Description Endorsement (SC_Chubb21120617.CBDE.1).pdf
COUNT 418
Design Exclusion (SC_GA_UMB_V3.DE.1).pdf
COUNT 419
Foreign Entity Loss Condition (SC_GA_EXCESS_CGL_V3.FELC.1).pdf
COUNT 420
Specified Insured Exclusion (SC_ GA_EXCESS_CGL_V3.SINY.1).pdf
COUNT 421
Contract Works Activities Endorsement (SC_Chubb02940217.CWAE.1).pdf
COUNT 422
Mining Operation Activities Exclusion (SC_Chubb02940217.MOAE.1).pdf
COUNT 423
Subsidence Exclusion (SC_MMA_CGL_V3.0_SUB.1).pdf
COUNT 424
Blood Exclusion (SC_Chu

...

Pollution (Named Perils) Exclusion (SC_BFAU1502_POLNMD.1).pdf
COUNT 1610
Audience Participation Exclusion (SC_UMB1502.AUPART.1).pdf
COUNT 1611
Batch Endorsement (SC_UMB1502.BCHE.1).pdf
COUNT 1612
Vehicles (Hazardous Goods) Exclusion (SC_AUMSL1504.VEHAZ.1).pdf
COUNT 1613
Labour Hire Exclusion (SC_MBFAU1711_LABOURE.1).pdf
COUNT 1614
GMO Exclusion (SC_AUMSL1504.GMO.1).pdf
COUNT 1615
Cyber and Data Breach Exclusion (SC_BFAU1502.CDBE.1).pdf
COUNT 1616
Errors and Omissions Endorsement (SC_BFEP1308.EO.1).pdf
COUNT 1617
Personal Injury to Contractors Underground (SC_UMB1502.PICUG.1).pdf
COUNT 1618
Change of Business Description Endorsement (SC_UMB1502.CBDE.1).pdf
COUNT 1619
Bushfire Exclusion (SC_EPA11-2016.BFE.1).pdf
COUNT 1620
Oil Industry Exclusion (SC_BFAU1502_OIE.1).pdf
COUNT 1621
TOBACCO Exclusion (SC_UMB1502.TOBE.2).pdf
COUNT 1622
Livestock Exclusion (SC_UMB1502.LIVESE.1).pdf
COUNT 1623
Total Communicable Disease Exclusion (SC_UMB1502.TCDE.1).pdf
COUNT 1624
Chubb16-250-1019 Chubb EBM Bu

Amusement Device Exclusion (SC_MBFAU1711_AMUSEDE.1).pdf
COUNT 1699
Total Communicable Disease Exclusion (SC_WBF02-102-0419.TCDE.1).pdf
COUNT 1700
Oil Industry Exclusion (SC_MBFAU1711_OIE.1).pdf
COUNT 1701
Change of Location Endorsement (SC_BFAU1502_CLOC.1).pdf
COUNT 1702
Aluminium Composite Panel Products Exclusion (SC_BFAU1502.ACPPE.1).pdf
COUNT 1703
Pure Financial Loss Exclusion (SC_UMB1502.PFLE.1).pdf
COUNT 1704
Participation (Specified Sports) Exclusion (SC_BFAU1502_PARSPE.1).pdf
COUNT 1705
Efficacy Endorsement (SC_UMB1502.EFFE.1).pdf
COUNT 1706
Marine and Marina Exclusion (SC_MBFAU1711_MM.1).pdf
COUNT 1707
Hazardous Goods Exclusion (SC_BFEP1308.HGE.1).pdf
COUNT 1708
Australian Standards Endorsement (SC_MBFAU1711_AUSE.1).pdf
COUNT 1709
Contract Works Activities Endorsement (SC_BFAU1502_CWAE.1).pdf
COUNT 1710
Demolition, Underpinning Exclusion (SC_EXLAU1502_DUPE.1).pdf
COUNT 1711
Pollution (Total) Exclusion (SC_AUMSL1504.POLT.1).pdf
COUNT 1712
Printers Errors and Omissions Endorseme

USA Canada Exports Costs Inclusive Endorsement (SC_AUMSL1504.USACE.1).pdf
COUNT 1818
Change of Insured Endorsement (SC_UMB1502.CIE.1).pdf
COUNT 1819
Professional Indemnity Exclusion - Total (SC_AUMSL1504.PIET.1).pdf
COUNT 1820
Property Owner Endorsement (SC_Chubb02-125-0218.POE.1).pdf
COUNT 1821
TOBACCO Exclusion (SC_EXLAU1502_TOBE.2).pdf
COUNT 1822
Blasting and Explosives Endorsement (SC_MBFAU1711_BLASTEE.1).pdf
COUNT 1823
Other Insurance (Protection and Indemnity) Exclusion (SC_EXLAU1502_OIPI.1).pdf
COUNT 1824
Formaldehyde Exclusion (SC_BFAU1502_FHE.1).pdf
COUNT 1825
Drug Use Exclusion (SC_EXLAU1502_DUE.1).pdf
COUNT 1826
Demolition, Underpinning Exclusion (SC_AUMSL1504.DUPE.1).pdf
COUNT 1827
Additional Insured Endorsement (SC_WBF02-102-0419.ADDIE.1).pdf
COUNT 1828
Silica Exclusion (SC_EXLAU1502_SE.1).pdf
COUNT 1829
Mining Limitation Exclusion (SC_BFAU1502_ML.1).pdf
COUNT 1830
Queensland Electrical Contractor Endorsement (SC_BFEP1308.QLDECE.1).pdf
COUNT 1831
DIL Endorsement (SC_WBF02-

Recourse Condition (SC_MBFAU1711_RC.1).pdf
COUNT 1907
Subsidence Exclusion (SC_MBFAU1711_SUB.1).pdf
COUNT 1908
Airside Liability Exclusion (SC_BFAU1502_AIRLE.1).pdf
COUNT 1909
Intoxication Exclusion (SC_MBFAU1711_INTOXE.1).pdf
COUNT 1910
Care Custody and Control (Other Property) Exclusion (SC_AUMSL1504.CCCE.1).pdf
COUNT 1911
Blood and Blood Bourne Products Exclusion (Write back for First Aid) (SC_BFAU1502_BBBPE.1).pdf
COUNT 1912
Treatment Risk Exclusion - Prescription (SC_UMB1502.TREX.1).pdf
COUNT 1913
Loss of  Electronic Data Exclusion (SC_MBFAU1711_LED.1).pdf
COUNT 1914
Separation of Insureds Endorsement (SC_MBFAU1711_SEPINS.1).pdf
COUNT 1915
Global DIC and DIL Endorsement (SC_MBFAU1711_GDICDIL.1).pdf
COUNT 1916
Animal Disease Exclusion (SC_BFEP1308.ADE.1).pdf
COUNT 1917
Cyber Exclusion (SC_AUMSL1504.CE.1).pdf
COUNT 1918
Spray Drift Exclusion (SC_EXLAU1502_SDE.1).pdf
COUNT 1919
Chubb Broadform Liability and Environmental Protect Quote Template.pdf
COUNT 1920
Demolition, Underpinning 

Insurance Arranged by the Principal (SC_WBF02-102-0419.IAP.1).pdf
COUNT 2008
Hazardous Goods Exclusion (SC_AUMSL1504.HGE.1).pdf
COUNT 2009
Energy (LSW 245) Exclusion (SC_AUMSL1504.LSW245.1).pdf
COUNT 2010
TOBACCO Exclusion (SC_BFEP1308.TOBE.2).pdf
COUNT 2011
Labour Hire Exclusion (SC_UMB1502.LABOURE.1).pdf
COUNT 2012
Molestation Endorsement (SC_AUMSL1504.ME.1).pdf
COUNT 2013
Mining Limitation (SC_EXLAU1502_ML.1).pdf
COUNT 2014
Extend Period Agreement (SC_AUMSL1504.EPA.1).pdf
COUNT 2015
Audience Participation Exclusion (SC_BFAU1502_AUPART.1).pdf
COUNT 2016
Cyber Exclusion (SC_BFAU1502.CE.1).pdf
COUNT 2017
Vehicles (Hazardous Goods) Exclusion (SC_UMB1502.VEHAZ.1).pdf
COUNT 2018
Specified Insured Exclusion (SC_BFAU1502_SINY.1).pdf
COUNT 2019
Organophosphates Exclusion (SC_MBFAU1711_OGPHO.1).pdf
COUNT 2020
Money Exclusion (SC_AUMSL1504.MONE.1).pdf
COUNT 2021
Amusement Device Exclusion (SC_EXLAU1502_AMUSEDE.1).pdf
COUNT 2022
Aircraft Charter Exclusion (SC_BFAU1502_AIRCE.1).pdf
COUNT 2023
Ca

In [23]:
len(local_indexed)

3017

In [24]:
from azure.cosmos import exceptions, CosmosClient, PartitionKey

cosmo_endpoint="https://nf-poc-cdb-sql.documents.azure.com"
cosmo_key="iEcEfrxYe0Fm9QtoxDrOpLvGsfzjowwybULlWT9Uz4XxV4RmOIAnRuLdgRFUu1LPU5Vwk3UGivRrPrxnk7083w=="
client = CosmosClient(cosmo_endpoint, cosmo_key)
database=client.get_database_client('policy-analysis')
container=database.get_container_client('casualty-wordings')

In [25]:
for key,value in local_indexed.items():
    doc={}
    doc['text']=key
    doc['page']=int(value[1])
    if value[4]==True or value[0].find('Exclusion')!=-1:
        doc['IsExclusion']=True
    else:
        doc['IsExclusion']=False
    doc['doc_name']=value[0].replace('.pdf','')
    doc['id']=doc['doc_name']+' '+str(doc['page'])
    doc['bold_phrases']=value[2]
    doc['plain_text']=value[3]
    doc['excl_count']=value[5]
    doc['excl_pos']=value[6]
    doc['folder']=value[7]
    doc['definitions']=[{'name':key,'text':value} for key,value in re.findall('###(.*?)@@(.*?)%%%',doc['text'])]
    doc['country']=value[9]
    doc['language']=value[10]
    doc['definition_text']=value[11]
#     print(doc['definition_text'],'\n\n')
    if doc['definition_text']:
        doc['definition_flag']=True
    else:
        doc['definition_flag']=False
    if value[14]==True or 'endorsement' in value[0].lower(): 
        doc['endorsements']=True
    else:
        doc['endorsements']=value[13]
    doc['excl_text']=value[14]
    doc['cond_text']=value[15]
    doc['cond_flag']=value[16]
    doc['cond_count']=value[17]
    doc['cond_pos']=value[18]
    doc['definitions_in_page']=value[19]
    doc['ext_text']=value[20]
    doc['ext_flag']=value[21]
    doc['ext_count']=value[22]
    doc['ext_pos']=value[23]
    
    try:       
        container.create_item(body=doc)
    except Exception as err:
        print(err)
        pass

In [26]:
print('added to cosmos')

added to cosmos
